In [27]:
import os
import numpy as np
import pandas as pd
from datetime import date
import datetime
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


DATA_ROOT = "./data/"
df_train = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
df_test = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))

In [28]:
# 轉換時間
def use_days(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

df_train['Date_received'] = pd.to_datetime(df_train['Date_received'], format='%Y%m%d')
df_train['Date'] = pd.to_datetime(df_train['Date'], format='%Y%m%d')
df_test['Date_received'] = pd.to_datetime(df_test['Date_received'], format='%Y%m%d')
df_train['use_days'] = (df_train.Date-df_train.Date_received).astype('timedelta64[D]')
df_train[['use_days']] = df_train[['use_days']].fillna(-1)

# 補距離遺失值
df_train[['Distance']] = df_train[['Distance']].fillna(99)
df_test[['Distance']] = df_test[['Distance']].fillna(99)

In [29]:
# 用戶直接消費
train_purch = df_train[pd.isnull(df_train.Date_received)]
test_purch = df_test[pd.isnull(df_test.Date_received)]

# 用戶取得優惠卷
train_received = df_train[pd.notnull(df_train.Date_received)]
test_received = df_test[pd.notnull(df_test.Date_received)]

In [30]:
# 建立label標籤
def label(row):
    if row['use_days'] < 0:
        return 0
    elif row['use_days'] < 16:
        return 1
    else:
        return 1
train_received["label"] = train_received.apply(label, axis=1)
train_received["label"].value_counts()

D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


0    704033
1     42936
Name: label, dtype: int64

In [31]:
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 'diff'
    else:
        return 'rate'

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountTotal(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountPrice(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    df['discount_total'] = df['Discount_rate'].astype('str').apply(getDiscountTotal)
    df['discount_price'] = df['Discount_rate'].astype('str').apply(getDiscountPrice)
    df['coupon_type'] = df[["discount_rate", "discount_type", "discount_total", "discount_price"]].apply(lambda x: ''.join(str(x.values)), axis=1)
    
    return df

train_received = processData(train_received)
test_received = processData(test_received)

D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anacoda3\lib\sit

In [32]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    return row.dayofweek+1 # add one to make it from 0~6 -> 1~7

train_received['weekday'] = train_received['Date_received'].apply(getWeekday)
test_received['weekday'] = test_received['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
train_received['weekday_type'] = train_received['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 )
test_received['weekday_type'] = test_received['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) 

D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anacoda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__

In [34]:
# disable
def get_the_same_counpon2(df):
    base_df = df_train[['User_id','Coupon_id','Date_received']]
    base_df = base_df.groupby(['User_id', 'Coupon_id', 'Date_received']).size().reset_index()[['User_id', 'Coupon_id', 'Date_received']]
    
    ba2_df = base_df.merge(base_df, on=['User_id','Coupon_id'])
    ba2_df = ba2_df[ba2_df.Date_received_x > ba2_df.Date_received_y]
    ba2_df['Date_received_x'] = ba2_df['Date_received_x'].astype(object)

    f1 = ba2_df.groupby(['User_id','Coupon_id','Date_received_x'])['Date_received_y'].count().reset_index()
    f1 = f1.rename(index=str, columns={"Date_received_y": "the_same_counpon_cnt"})
    f2 = ba2_df.groupby(['User_id','Coupon_id','Date_received_x'])['Date_received_y'].max().reset_index()
    f1['Date_received_x'] = f1['Date_received_x'].astype(object)
    f2['Date_received_x'] = f2['Date_received_x'].astype(object)

    ba2_df = ba2_df.merge(f2, how='left', left_on=['User_id','Coupon_id','Date_received_x'], right_on=['User_id','Coupon_id','Date_received_x'])
    ba2_df = ba2_df.merge(f1, how='left', left_on=['User_id','Coupon_id','Date_received_x'], right_on=['User_id','Coupon_id','Date_received_x'])

    ba2_df['coupon_get_last_days'] = (pd.to_datetime(ba2_df['Date_received_x']) - pd.to_datetime(ba2_df['Date_received_y_x'])).astype('timedelta64[D]')
    ba2_df = ba2_df.rename(index=str, columns={"Date_received_x": "Date_received"})
    

    return ba2_df

In [35]:
# disable
def get_coupon_from(df_train, df_purch, df_test):
    base_df = pd.concat([df_train[['User_id','Merchant_id','coupon_type','Date_received']],
                    df_test[['User_id','Merchant_id','coupon_type','Date_received']]])
    base_df = base_df.groupby(['User_id', 'Merchant_id', 'Date_received']).size().reset_index()[['User_id', 'Merchant_id', 'Date_received']]
    ref_df = df_train[['User_id','Merchant_id','coupon_type','Date_received','use_days' ,'Date']]
    ref_df = ref_df[ref_df.Date.notna()]
    tmp_df = df_purch[['User_id','Merchant_id','Coupon_id','Date_received','use_days' ,'Date']]
    tmp_df = tmp_df.rename(index=str, columns={"Coupon_id": "coupon_type"})
    tmp_df = tmp_df[tmp_df.Date.notna()]
    ref_df = pd.concat([ref_df,tmp_df])
    base_df['Date_received'] = base_df['Date_received'].astype(object)
    ref_df['Date'] = ref_df['Date'].astype(object)
    base_df = base_df.merge(ref_df,how='inner', left_on=['User_id','Merchant_id','Date_received'], right_on=['User_id','Merchant_id','Date'] )
    base_df = base_df.rename(index=str, columns={"Date_received_x": "Date_received"})

    m1 = base_df.groupby(['User_id','Merchant_id','Date_received'])['use_days'].count().reset_index()
    m1 = m1.rename(index=str, columns={"use_days": "counpon_from_purch_cnt"})

    return m1

In [36]:
def get_user_coupon_sta(df_train, df_test):
    d1 = pd.concat([df_train[['User_id','Merchant_id','coupon_type','Date_received']],
                    df_test[['User_id','Merchant_id','coupon_type','Date_received']]])
    df1 = df_train[['User_id','Merchant_id','coupon_type','Date_received','label','use_days']]
    df1 = d1.merge(df1, how='left', left_on=['User_id','coupon_type'], right_on=['User_id','coupon_type'])
    df1 = df1[df1.Date_received_x > df1.Date_received_y]
    dd1 = df1.groupby(['User_id','coupon_type','Date_received_x'])['label'].count().reset_index()
    dd2 = df1[df1.label == 1].groupby(['User_id','coupon_type','Date_received_x'])['label'].count().reset_index()
    dd3 = df1[df1.label == 1].groupby(['User_id','coupon_type','Date_received_x'])['use_days'].min().reset_index()
    dd1 = dd1.merge(dd2, how='left', left_on=['User_id','coupon_type','Date_received_x'], right_on=['User_id','coupon_type','Date_received_x'])
    dd1 = dd1.fillna(0)
    dd1 = dd1.merge(dd3, how='left', left_on=['User_id','coupon_type','Date_received_x'], right_on=['User_id','coupon_type','Date_received_x'])
    dd1 = dd1.fillna(1000)
    dd1['user_coupon_use_ratio'] = dd1['label_y']/dd1['label_x']
    dd1 = dd1.rename(index=str, columns={"Date_received_x": "Date_received", "label_x": "user_total_coupon"
                                         , "label_y": "user_use_coupon", "use_days":"user_coupon_mean_days"})
    
    return dd1

In [55]:
# disable
def get_coupon_number(df_train, df_test):   
    
    def coupon_total_cnt(row):
        coupon_type = row['coupon_type']
        Date_received = row['Date_received']
        x = df1[(df1.coupon_type==coupon_type) & (df1.Date_received<Date_received)].shape[0]
        return x

    def coupon_use_cnt(row):
        coupon_type = row['coupon_type']
        Date_received = row['Date_received']
        x = df1[(df1.label == 1) & (df1.coupon_type==coupon_type) & (df1.Date_received<Date_received)].shape[0]
        return x 
    
    d1 = pd.concat([df_train[['User_id','Merchant_id','coupon_type','Date_received']],
                    df_test[['User_id','Merchant_id','coupon_type','Date_received']]])
    d1 = d1.groupby(['coupon_type','Date_received']).size().reset_index()[['coupon_type','Date_received']]
    df1 = df_train[['User_id','Merchant_id','coupon_type','Date_received','label','use_days']]

    
    d1['coupon_total_cnt'] = d1.apply(coupon_total_cnt, axis=1)
    d1['coupon_use_cnt'] = d1.apply(coupon_use_cnt, axis=1)

    d1['coupon_ratio'] = d1['coupon_use_cnt'] / d1['coupon_total_cnt']
    d1 = d1.fillna(0)

    return d1

In [56]:
def get_user_number(df_train, df_test):
    d1 = pd.concat([df_train[['User_id','Merchant_id','coupon_type','Date_received']],
                    df_test[['User_id','Merchant_id','coupon_type','Date_received']]])
    df1 = df_train[['User_id','Merchant_id','coupon_type','Date_received','label','use_days']]
    df1 = d1.merge(df1, how='left', left_on=['User_id'], right_on=['User_id'])
    df1 = df1[df1.Date_received_x > df1.Date_received_y]

    dd1 = df1.groupby(['User_id','Date_received_x'])['label'].count().reset_index()
    dd2 = df1[df1.label == 1].groupby(['User_id','Date_received_x'])['label'].count().reset_index()
    dd3 = df1[df1.label == 1].groupby(['User_id','Date_received_x'])['use_days'].min().reset_index()
    dd1 = dd1.merge(dd2, how='left', left_on=['User_id','Date_received_x'], right_on=['User_id','Date_received_x'])
    dd1 = dd1.fillna(0)
    dd1 = dd1.merge(dd3, how='left', left_on=['User_id','Date_received_x'], right_on=['User_id','Date_received_x'])
    dd1 = dd1.fillna(1000)
    dd1['acoupon_ratio'] = dd1['label_y']/dd1['label_x']
    dd4 = df1.groupby(['User_id','Date_received_x'])['Date_received_y'].min().reset_index()
    dd1 = dd1.merge(dd4, how='left', left_on=['User_id','Date_received_x'], right_on=['User_id','Date_received_x'])
    dd1['coupon_use_last_days'] = (pd.to_datetime(dd1['Date_received_x']) - pd.to_datetime(dd1['Date_received_y'])).astype('timedelta64[D]')
    dd1 = dd1.rename(index=str, columns={"Date_received_x": "Date_received", "label_x": "auser_total_cnt"
                                         , "label_y": "auser_use_cnt", "use_days":"user_mean_days", "Date_received_y":"coupon_use_last_date"})

    return dd1

In [57]:
def get_merchant_coupon_number(df):
    
    base = df[['User_id','Merchant_id','coupon_type','Date_received']]
    f1 = base.groupby(['Merchant_id'])['User_id'].count().reset_index()
    f1 = f1.rename(index=str, columns={'User_id':'send_coupon_cnt'})
    
    return f1

In [58]:
def get_user_coupon_number(df):
    
    base = df[['User_id','Merchant_id','coupon_type','Date_received']]
    f1 = base.groupby(['User_id'])['Merchant_id'].count().reset_index()
    f1 = f1.rename(index=str, columns={'Merchant_id':'user_recieved_coupon_cnt'})
    
    return f1

In [59]:
def get_coupon_id_number(df):
    
    base = df[['User_id','Merchant_id','Coupon_id','Date_received']]
    f1 = base.groupby(['User_id', 'Coupon_id'])['Merchant_id'].count().reset_index()
    f1 = f1.rename(index=str, columns={'Merchant_id':'coupon_id_recieved_coupon_cnt'})
    
    return f1

In [60]:
def get_user_purch_number(df):
    
    base = df[['User_id','Merchant_id','Coupon_id','Date_received']]
    f1 = base.groupby(['User_id'])['Merchant_id'].count().reset_index()
    f1 = f1.rename(index=str, columns={'Merchant_id':'user_purch_cnt'})
    
    return f1

In [61]:
###### 我是分隔線 #######
#pretrain_received = pd.read_pickle("./pretrain_received.pkl")
#valid_received = pd.read_pickle("./valid_received.pkl")

In [62]:
# 建立feature of pretrain and valid data
valid_date = '20160416'
pretrain_received = train_received[train_received.Date_received < pd.to_datetime(valid_date, format="%Y%m%d")]
valid_received = train_received[train_received.Date_received >= pd.to_datetime(valid_date, format="%Y%m%d")]
pretrain_purch = train_purch[train_purch.Date < pd.to_datetime(valid_date, format="%Y%m%d")]
valid_purch = train_purch[train_purch.Date >= pd.to_datetime(valid_date, format="%Y%m%d")]

In [63]:
user_cnt = get_user_number(pretrain_received, valid_received)
pretrain_received = pretrain_received.merge(user_cnt, how='left', left_on=['User_id','Date_received'], right_on=['User_id','Date_received'])
valid_received = valid_received.merge(user_cnt, how='left', left_on=['User_id','Date_received'], right_on=['User_id','Date_received'] )

In [64]:
user_coupon_cnt = get_user_coupon_sta(pretrain_received, valid_received)
pretrain_received = pretrain_received.merge(user_coupon_cnt, how='left', left_on=['User_id','coupon_type','Date_received'], right_on=['User_id','coupon_type','Date_received'])
valid_received = valid_received.merge(user_coupon_cnt, how='left', left_on=['User_id','coupon_type','Date_received'], right_on=['User_id','coupon_type','Date_received'] )

In [65]:
merchant_cnt = get_merchant_coupon_number(pretrain_received)
pretrain_received = pretrain_received.merge(merchant_cnt, how='left', on = 'Merchant_id')
merchant_cnt = get_merchant_coupon_number(valid_received)
valid_received = valid_received.merge(merchant_cnt, how='left', on = 'Merchant_id')

In [66]:
user_cnt = get_user_coupon_number(pretrain_received)
pretrain_received = pretrain_received.merge(user_cnt, how='left', on = 'User_id')
user_cnt = get_user_coupon_number(valid_received)
valid_received = valid_received.merge(user_cnt, how='left', on = 'User_id')

In [67]:
coupon_cnt = get_coupon_id_number(pretrain_received)
pretrain_received = pretrain_received.merge(coupon_cnt, how='left', on = ['User_id','Coupon_id'])
coupon_cnt = get_coupon_id_number(valid_received)
valid_received = valid_received.merge(coupon_cnt, how='left', on = ['User_id','Coupon_id'])

In [68]:
purch_cnt = get_user_purch_number(pretrain_purch)
pretrain_received = pretrain_received.merge(purch_cnt, how='left', on = ['User_id'])
purch_cnt = get_user_purch_number(valid_purch)
valid_received = valid_received.merge(purch_cnt, how='left', on = ['User_id'])
pretrain_received = pretrain_received.fillna(0)
valid_received = valid_received.fillna(0)

In [100]:
pretrain_received.to_pickle("./pretrain_received.pkl")
valid_received.to_pickle("./valid_received.pkl")

In [69]:
pretrain_received.corr()['label'].sort_values(ascending=False)

label                            1.000000
use_days                         0.718476
user_coupon_use_ratio            0.256518
acoupon_ratio                    0.229249
user_purch_cnt                   0.224103
coupon_id_recieved_coupon_cnt    0.218260
auser_use_cnt                    0.132325
user_use_coupon                  0.131705
user_recieved_coupon_cnt         0.107545
user_total_coupon                0.100418
coupon_use_last_days             0.097542
auser_total_cnt                  0.071877
Coupon_id                        0.011965
Merchant_id                      0.011054
User_id                         -0.000146
user_coupon_mean_days           -0.003805
user_mean_days                  -0.003963
weekday                         -0.012617
Distance                        -0.020537
send_coupon_cnt                 -0.081829
discount_rate                   -0.089685
discount_price                  -0.127560
discount_total                  -0.144074
weekday_type                      

In [70]:
pretrain_received.columns

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'use_days', 'label', 'discount_rate',
       'discount_type', 'discount_total', 'discount_price', 'coupon_type',
       'weekday', 'weekday_type', 'auser_total_cnt', 'auser_use_cnt',
       'user_mean_days', 'acoupon_ratio', 'coupon_use_last_date',
       'coupon_use_last_days', 'user_total_coupon', 'user_use_coupon',
       'user_coupon_mean_days', 'user_coupon_use_ratio', 'send_coupon_cnt',
       'user_recieved_coupon_cnt', 'coupon_id_recieved_coupon_cnt',
       'user_purch_cnt'],
      dtype='object')

In [76]:
###### 我是分隔線 #########
# 建立feature of train and test data

In [89]:
user_cnt = get_user_number(train_received, test_received)
train_received_t = train_received.merge(user_cnt, how='left', left_on=['User_id','Date_received'], right_on=['User_id','Date_received'])
test_received_t = test_received.merge(user_cnt, how='left', left_on=['User_id','Date_received'], right_on=['User_id','Date_received'] )

In [90]:
user_coupon_cnt = get_user_coupon_sta(train_received, test_received)
train_received_t = train_received_t.merge(user_coupon_cnt, how='left', left_on=['User_id','coupon_type','Date_received'], right_on=['User_id','coupon_type','Date_received'])
test_received_t = test_received_t.merge(user_coupon_cnt, how='left', left_on=['User_id','coupon_type','Date_received'], right_on=['User_id','coupon_type','Date_received'] )

In [92]:
merchant_cnt = get_merchant_coupon_number(train_received_t)
train_received_t = train_received_t.merge(merchant_cnt, how='left', on = 'Merchant_id')
merchant_cnt = get_merchant_coupon_number(test_received_t)
test_received_t = test_received_t.merge(merchant_cnt, how='left', on = 'Merchant_id')

In [93]:
user_cnt = get_user_coupon_number(train_received_t)
train_received_t = train_received_t.merge(user_cnt, how='left', on = 'User_id')
user_cnt = get_user_coupon_number(test_received_t)
test_received_t = test_received_t.merge(user_cnt, how='left', on = 'User_id')

In [94]:
coupon_cnt = get_coupon_id_number(train_received_t)
train_received_t = train_received_t.merge(coupon_cnt, how='left', on = ['User_id','Coupon_id'])
coupon_cnt = get_coupon_id_number(test_received_t)
test_received_t = test_received_t.merge(coupon_cnt, how='left', on = ['User_id','Coupon_id'])

In [95]:
purch_cnt = get_user_purch_number(train_purch)
train_received_t = train_received_t.merge(purch_cnt, how='left', on = ['User_id'])
purch_cnt = get_user_purch_number(test_purch)
test_received_t = test_received_t.merge(purch_cnt, how='left', on = ['User_id'])

In [99]:
train_received_t.to_pickle("./train_received_t.pkl")
test_received_t.to_pickle("./test_received_t.pkl")